In [1]:
# mnist dataset is already heavily preprocessed. Just scale & split
# required.

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf  # for creation & training of nn
import tensorflow_datasets as tfds   # to obtain mnist dataset
from tensorboard.plugins.hparams import api as hp


Downloading & Preprocessing data # this step has nothing to do with tensorboard

In [29]:
# Parameters to be used later on-
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [30]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised=True)

In [31]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

In [32]:
# In ML we like no's that are standardised in some way. So  technique in images is to scale pixel values b/w 0 and 1 which originally would be b/w 0 and 255 for greyscale.
# Divide all pixels in dataset by 255, so value will be b/w 0 & 1.

In [33]:
mnist_train

<PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [34]:
mnist_test

<PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [35]:
def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /= 255.

    return image, label

In [36]:
# Scale every image in train and test dataset
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [37]:
# Validation set is crucial to prevent overfitting using early stopping
# Split training set manually to create a validation set (10% of
# training set)

In [38]:
mnist_info.splits['train'].num_examples

60000

In [39]:
mnist_info.splits['test'].num_examples

10000

In [40]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

In [41]:
num_validation_samples

6000.0

In [42]:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [43]:
# we need validation set to contain data with same distribution as training data

In [44]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [45]:
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [46]:
# Test Data
num_test_samples = mnist_info.splits['test'].num_examples

In [47]:
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [48]:
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [49]:
# batch dataset for optimal performance of network
# batch size generally recommended in power of 2 (32,64,128) etc

# validation & test sets not necessarily be batched as we do not
# backward propagate on them however model expects them to be batched to get proper dimensions.

In [50]:
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

Defining Hyperparameters

In [51]:
# we want to tune 2 hyperparameters
# 1. kernel size (3*3, 5*5, 7*7)
# 2. optimizer (ADAM/SGD)

In [52]:
# 6 combinations so 6 times model need to be trained

HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE,HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

Build, Train and Test Network

In [53]:
# Define configuration of n/w
# Compile model
# Fit model on training data

In [54]:
def train_test_model(hparams):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,hparams[HP_FILTER_SIZE],activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50,hparams[HP_FILTER_SIZE],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
    ])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss=loss_fn, metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
    )
    model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    # earlystopping callback should always be last element of list
    callbacks = [early_stopping],
    validation_data = validation_data,
    verbose = 2 # print info at end of each epoch
    )
    _, accuracy = model.evaluate(test_data)

    return accuracy

In [55]:
def run(log_dir, hparams):

    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams) # record values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [56]:
# to iterate over every combination of hyperparameter
session_num = 0
for filter_size in HP_FILTER_SIZE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        # dictionary with current values
        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_OPTIMIZER: optimizer
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)

        session_num +=1

--- Starting trial: run-0
{'filter_size': 3, 'optimizer': 'adam'}
Epoch 1/20
422/422 - 21s - loss: 0.2863 - accuracy: 0.9191 - val_loss: 0.0956 - val_accuracy: 0.9717 - 21s/epoch - 49ms/step
Epoch 2/20
422/422 - 20s - loss: 0.0765 - accuracy: 0.9774 - val_loss: 0.0487 - val_accuracy: 0.9862 - 20s/epoch - 48ms/step
Epoch 3/20
422/422 - 20s - loss: 0.0543 - accuracy: 0.9833 - val_loss: 0.0428 - val_accuracy: 0.9875 - 20s/epoch - 48ms/step
Epoch 4/20
422/422 - 20s - loss: 0.0456 - accuracy: 0.9862 - val_loss: 0.0401 - val_accuracy: 0.9873 - 20s/epoch - 48ms/step
Epoch 5/20
422/422 - 20s - loss: 0.0382 - accuracy: 0.9888 - val_loss: 0.0335 - val_accuracy: 0.9908 - 20s/epoch - 48ms/step
Epoch 6/20
422/422 - 21s - loss: 0.0323 - accuracy: 0.9903 - val_loss: 0.0330 - val_accuracy: 0.9898 - 21s/epoch - 49ms/step
Epoch 7/20
422/422 - 21s - loss: 0.0290 - accuracy: 0.9912 - val_loss: 0.0243 - val_accuracy: 0.9923 - 21s/epoch - 49ms/step
Epoch 8/20
422/422 - 21s - loss: 0.0262 - accuracy: 0.9921 

Visualizations in Tensorboard

In [57]:
# Start tensorboard application
%load_ext tensorboard

In [58]:
%tensorboard --logdir "logs/hparam_tuning"

Launching TensorBoard...